## Build the rgrid maps and reproject the FUV maps onto W4 headers (for the SFR tracer)

In [ ]:
import numpy as np
import pandas as pd
import math
import os.path
sys.path.append(os.path.join(os.getenv('HOME'),'workspace','galbase'))
from gal_data import gal_data
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astropy.utils.console import ProgressBar
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from operator import itemgetter
from astropy.coordinates import SkyCoord
import astropy.units as u
from reproject import reproject_interp

In [ ]:
galbase = pd.read_csv('samples/galbase_info.csv')
samp = pd.read_csv('samples/sne_sample.csv')

In [ ]:
from __future__ import (
    division, print_function, absolute_import, unicode_literals)


def deproject(center_coord=None, incl=0*u.deg, pa=0*u.deg,
              header=None, wcs=None, naxis=None, ra=None, dec=None,
              return_offset=False):

    """
    Calculate deprojected radii and projected angles in a disk.
    This function deals with projected images of astronomical objects
    with an intrinsic disk geometry. Given sky coordinates of the
    disk center, disk inclination and position angle, this function
    calculates deprojected radii and projected angles based on
    (1) a FITS header (`header`), or
    (2) a WCS object with specified axis sizes (`wcs` + `naxis`), or
    (3) RA and DEC coodinates (`ra` + `dec`).
    Both deprojected radii and projected angles are defined relative
    to the center in the inclined disk frame. For (1) and (2), the
    outputs are 2D images; for (3), the outputs are arrays with shapes
    matching the broadcasted shape of `ra` and `dec`.
    Parameters
    ----------
    center_coord : `~astropy.coordinates.SkyCoord` object or 2-tuple
        Sky coordinates of the disk center
    incl : `~astropy.units.Quantity` object or number, optional
        Inclination angle of the disk (0 degree means face-on)
        Default is 0 degree.
    pa : `~astropy.units.Quantity` object or number, optional
        Position angle of the disk (red/receding side, North->East)
        Default is 0 degree.
    header : `~astropy.io.fits.Header` object, optional
        FITS header specifying the WCS and size of the output 2D maps
    wcs : `~astropy.wcs.WCS` object, optional
        WCS of the output 2D maps
    naxis : array-like (with two elements), optional
        Size of the output 2D maps
    ra : array-like, optional
        RA coordinate of the sky locations of interest
    dec : array-like, optional
        DEC coordinate of the sky locations of interest
    return_offset : bool, optional
        Whether to return the angular offset coordinates together with
        deprojected radii and angles. Default is to not return.
    Returns
    -------
    deprojected coordinates : list of arrays
        If `return_offset` is set to True, the returned arrays include
        deprojected radii, projected angles, as well as angular offset
        coordinates along East-West and North-South direction;
        otherwise only the former two arrays will be returned.
    Notes
    -----
    This is the Python version of an IDL function `deproject` included
    in the `cpropstoo` package. See URL below:
    https://github.com/akleroy/cpropstoo/blob/master/cubes/deproject.pro
    """

    if isinstance(center_coord, SkyCoord):
        x0_deg = center_coord.ra.degree
        y0_deg = center_coord.dec.degree
    else:
        x0_deg, y0_deg = center_coord
        if hasattr(x0_deg, 'unit'):
            x0_deg = x0_deg.to(u.deg).value
            y0_deg = y0_deg.to(u.deg).value
    if hasattr(incl, 'unit'):
        incl_deg = incl.to(u.deg).value
    else:
        incl_deg = incl
    if hasattr(pa, 'unit'):
        pa_deg = pa.to(u.deg).value
    else:
        pa_deg = pa

    if header is not None:
        wcs_cel = WCS(header).celestial
        naxis1 = header['NAXIS1']
        naxis2 = header['NAXIS2']
        # create ra and dec grids
        ix = np.arange(naxis1)
        iy = np.arange(naxis2).reshape(-1, 1)
        ra_deg, dec_deg = wcs_cel.wcs_pix2world(ix, iy, 0)
    elif (wcs is not None) and (naxis is not None):
        wcs_cel = wcs.celestial
        naxis1, naxis2 = naxis
        # create ra and dec grids
        ix = np.arange(naxis1)
        iy = np.arange(naxis2).reshape(-1, 1)
        ra_deg, dec_deg = wcs_cel.wcs_pix2world(ix, iy, 0)
    else:
        ra_deg, dec_deg = np.broadcast_arrays(ra, dec)
        if hasattr(ra_deg, 'unit'):
            ra_deg = ra_deg.to(u.deg).value
            dec_deg = dec_deg.to(u.deg).value

    # recast the ra and dec arrays in term of the center coordinates
    # arrays are now in degrees from the center
    dx_deg = (ra_deg - x0_deg) * np.cos(np.deg2rad(y0_deg))
    dy_deg = dec_deg - y0_deg

    # rotation angle (rotate x-axis up to the major axis)
    rotangle = np.pi/2 - np.deg2rad(pa_deg)

    # create deprojected coordinate grids
    deprojdx_deg = (dx_deg * np.cos(rotangle) +
                    dy_deg * np.sin(rotangle))
    deprojdy_deg = (dy_deg * np.cos(rotangle) -
                    dx_deg * np.sin(rotangle))
    deprojdy_deg /= np.cos(np.deg2rad(incl_deg))

    # make map of deprojected distance from the center
    radius_deg = np.sqrt(deprojdx_deg**2 + deprojdy_deg**2)

    # make map of angle w.r.t. position angle
    projang_deg = np.rad2deg(np.arctan2(deprojdy_deg, deprojdx_deg))

    if return_offset:
        return radius_deg, projang_deg, dx_deg, dy_deg
    else:
        return radius_deg, projang_deg

Function that will take the wcs of a FITS image and will build galactocentric radii maps (in degrees).

In [ ]:
def build_rgrid(wcs, ra_gal, dec_gal, incl_gal, pa_gal):
    """ Input: WCS of the fits image for a galaxy
        Output: a grid of galactocentric radii for each pixel in the galaxy image
    """
    naxis = wcs._naxis # grab the axes
    
    # make an map 'x' for pixels along the RA axis and a map 'y' for pixels along the DEC axis
    x, y = np.meshgrid(range(naxis[0]), range(naxis[1]))
    
    # convert each x and y map to RA and DEC maps in degrees
    ra_deg, dec_deg = wcs.wcs_pix2world(x,y, 0)
    
    rgal_map, phigal_map = deproject(center_coord=(ra_gal, dec_gal), incl=incl_gal, pa=pa_gal, wcs=wcs, naxis=naxis)
    
    return(rgal_map)

### Build the rgrid maps

Output the rgrid maps. Manually change the filename if for w4 to 'rgrid/%s/%s_%s_w4_rgrid.fits'% (res,pgc,res)

In [ ]:
def output_rgrid_maps(samp, res, w4):
    """ Input: sample of SNe and resolution (string)
        Output: Maps of galactocentric radii for galaxies as FITS files
    """
    
    bar = ProgressBar(len(samp), ipython_widget=True)
    for index, row in samp.iterrows():
        bar.update()
        pgc = row['PGC']
        
        # skip if rgrid map is already built for this galaxy; pick up where we left off
        if w4 is not True:
            if os.path.isfile('rgrid/%s/%s_%s_rgrid.fits'% (res,pgc,res)) is True:
                continue
        else:
            if os.path.isfile('rgrid/%s/%s_%s_w4_rgrid.fits'% (res,pgc,res)) is True:
                continue

        # get important information about the galaxy    
        gal_info = galbase[galbase['PGC'] == pgc]
        ra_gal   = gal_info['RA'].values
        dec_gal  = gal_info['DEC'].values
        incl_gal = gal_info['INCL'].values
        pa_gal   = gal_info['PA'].values
        r25_deg  = gal_info['R25'].values

        # open the convolved map for this galaxy
        if w4 is not True:
            try:
                hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_w1_%s.fits' % (res, pgc, res))
            except:
                continue
        else:
            try:
                hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_w4_%s.fits' % (res, pgc, res))
            except:
                continue
            
        wcs = WCS(hdulist[0].header)
        hdulist.close()

        # build rgrid for this galaxy
        rgrid = build_rgrid(wcs, ra_gal, dec_gal, incl_gal, pa_gal)

        # write to a FITS file
        hdu = pyfits.PrimaryHDU(rgrid)
        
        if w4 is not True:
            hdu.writeto('rgrid/%s/%s_%s_rgrid.fits'% (res,pgc,res), overwrite=True)
        else:
            hdu.writeto('rgrid/%s/%s_%s_w4_rgrid.fits'% (res,pgc,res), overwrite=True)
    return

In [ ]:
output_rgrid_maps(samp, '2kpc',  False)
output_rgrid_maps(samp, '2kpc', True)

### Reproject FUV onto W4 images for the SFR tracer

Reproject the FUV and W4 images so that we can create the SFR tracer. Reproject to the one with the smallest pixels (so reproject FUV onto the W4 header). Output reprojected FUV image as a FITS file.

In [ ]:
def reproject_fuv_on_w4(samp, res):
    """input: sample, resolutions
       output: FUV images with reprojected headers
       
       This will allow for us to build the SFR tracer.
    """
    
    bar = ProgressBar(len(samp), ipython_widget=True)
    for index, row in samp.iterrows():
        pgc = row['PGC']
        
        if os.path.isfile('rgrid/fuv_on_w4_%s/%s_fuv_on_w4_%s.fits'% (res,pgc,res)) is True:
            continue
        
        try:
            hdulist_fuv = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_fuv_%s.fits' % (res,pgc,res))
            hdulist_w4  = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_w4_%s.fits'  % (res,pgc,res))
            
        except FileNotFoundError:
            bar.update()
            continue
            
        # return array with reprojected pixels, footprint with information about original image
        array, footprint = reproject_interp(hdulist_fuv, hdulist_w4[0].header)
        pyfits.writeto('rgrid/fuv_on_w4_%s/%s_fuv_on_w4_%s.fits' % (res,pgc,res), array, 
                       hdulist_w4[0].header, overwrite=True)

        bar.update()
    return

In [ ]:
reproject_fuv_on_w4(samp, '2kpc')